# Analyze ladder pilot counts in DESeq2

In [ ]:
library("data.table")
library("dplyr")
library("DESeq2")
# for parallelization
library("BiocParallel")
register(MulticoreParam(8))

setwd("/home/users/rang/crispey3/ladder_pilot_feb2021/")

## Setup path to files

In [ ]:
# samples info file
samples_info_file = "./ladder_pilot_samples.txt"
# counts file
counts_file = "./counts/pbc_counts.txt"

# output directory
dir.create("./deseq/")
# deseq results file
deseq_output_file= "./deseq/ladder_pilot_deseq_results.txt"

print("Input files:")
print(samples_info_file)
print(counts_file)
print("Output files:")
print(deseq_output_file)

## Load samples info, barcode counts

In [ ]:
# loading samples
samples <- read.table(file = samples_info_file, header = TRUE, sep = '\t', row.names = 1)
samples$rep = factor(samples$rep)

# loading counts
counts <- fread(counts_file, header=TRUE, sep = '\t', na.strings="", data.table=TRUE)
colnames(counts) <- c('barcode', rownames(samples))

## Run DESeq with default settings to obtain barcode logFC values in CRISPEY competition
Assumes all samples are biological replicates of the same competition setup

In [ ]:
# Default DESeq workflow
dds <- DESeqDataSetFromMatrix(countData = as.matrix(counts, rownames = 1),
                              colData = samples,
                              design= ~gen+rep)
dds

# filter barcodes that do not meet minimum read cutoff
total_reads_cutoff = 1000
keep <- rowSums(counts(dds)) >= total_reads_cutoff
dds <- dds[keep,]

# run DEseq on default settings
dds <- DESeq(dds)

In [ ]:
# view results summary
res <- results(dds, name='gen', alpha=0.05)
summary(res)

# view table of results sorted by pvalue
resOrdered <- res[order(res$pvalue),]
resOrdered

# export results to file
write.csv(as.data.frame(resOrdered), file=deseq_output_file)

## To measure fitness relative to neutral barcodes, specify neutral set in controlGenes when estimating size factors and recalculate logFC

In [ ]:
ladder_oligos_annotation_file <- 'ladder_oligos_annotated.txt'
ladder_oligos_annotation <- fread(file = ladder_oligos_annotation_file, header = TRUE, sep = '\t', data.table=FALSE)

# get neutral barcode set
neutral_barcodes = as.vector(ladder_oligos_annotation[ladder_oligos_annotation$breslow_growth_rate==1,'PBC'])
neutral_barcodes

In [ ]:
# re-run DESeq with specified neutral set
dds <- DESeqDataSetFromMatrix(countData = as.matrix(counts, rownames = 1),
                              colData = samples,
                              design= ~gen+rep)

# filter barcodes that do not meet minimum read cutoff
total_reads_cutoff = 1000
keep <- rowSums(counts(dds)) >= total_reads_cutoff
dds <- dds[keep,]

# estimate size factors, specifying neutral barcodes under controlGenes
dds <- estimateSizeFactors(dds, controlGenes=grep(paste(neutral_barcodes, collapse="|"), rownames(dds)))
# estimate dispersion
dds <- estimateDispersions(dds)
# calculate logFC
dds <- nbinomWaldTest(dds)

In [ ]:
# view results summary
res <- results(dds, name='gen', alpha=0.05)
summary(res)

# view table of results sorted by pvalue
resOrdered <- res[order(res$pvalue),]
resOrdered

# export results to file
write.csv(as.data.frame(resOrdered), file=deseq_output_file)

## To measure the effect of a treatment on fitness effect, add the treatment variable to the dds design and re-run DEseq
New design should include the <code>gen\*cond</code> and <code>rep\*cond</code> interaction terms.<br>
Ensure that the samples file contains the <code>cond</code> column indicating the treatment status of each sample.<br>
Pseudo-conditions can be created between competition replicates to test for possible variance between competition batches (e.g. rep123 vs rep456). logFC values calculated this way should be the same as when each condition's samples is run in DESeq2 separately.

In [ ]:
# samples file with condition label
samples_cond_file <- "./ladder_pilot_samples_pseudoConditions.txt"
# loading samples
samples_cond <- read.table(file = samples_cond_file, header = TRUE, sep = '\t', row.names = 1)
samples_cond$rep = factor(samples_cond$rep)

In [ ]:
# re-run DESeq
dds2 <- DESeqDataSetFromMatrix(countData = as.matrix(counts, rownames = 1),
                                         colData = samples_cond,
                                         design= ~gen+rep+cond+gen:cond+rep:cond)

# filter barcodes that do not meet minimum read cutoff
total_reads_cutoff = 1000

keep <- rowSums(counts(dds2)) >= total_reads_cutoff
dds2 <- dds2[keep,]

# estimate size factors, specifying neutral oligos as control genes
dds2 <- estimateSizeFactors(dds2, controlGenes=grep(paste(neutral_barcodes, collapse="|"), rownames(dds2)))
# estimate dispersion
dds2 <- estimateDispersions(dds2)
# calculate logFC
dds2 <- nbinomWaldTest(dds2)

## To view condition-specific fitness, specify contrast when viewing results
If multiple conditions are present, fitness changes between non-reference conditions can be compared directly by specifying: <code>results(dds2, contrast=list("gen.condC", "gen.condB"))</code>

In [ ]:
# view all terms in deseq model
resultsNames(dds2)

# view results:
### fitness in cond A (control)
res2 <- results(dds2, name='gen')
summary(res2)

### fitness in cond B (treatment)
res3 <- results(dds2, contrast=list( c("gen","gen.condB") ))
summary(res3)

### changes in fitness between cond A and B
res4 <- results(dds2, name='gen.condB')
summary(res4)

# for fitness in cond A, view table of results sorted by pvalue
resOrdered2 <- res2[order(res2$pvalue),]
resOrdered2

# # export results to file
# write.csv(as.data.frame(resOrdered2), file="deseq_fitness_condA.txt")